# Fine Tuning Pre-trained WAV2VEC Using DSing Train

## Summary

1. Loading WAV2VEC model
2. Creating a Huggingface dataset for batch processing by Cropping the DSing Dataset described below (8 songs for each split, 1.5seconds max)

The dataset is cropped to allow for processing examples for this notebook.  in the actual processing, it will look different.

## DSing Dataset

Reading data from DSing Dataset.  Filesystem formatted this way to convert easily to huggingface dataset.

where: <br>
dev/test/trainX are datasets split.<br>
\[split\]_text contains transcript for the snippet.<br>
\[split\]_spk2gender contains information about gender for snippet.<br>

Tests Split: 480 Utterances, 48 minutes<br>
Dev Split: 482 Utterances, 41 minutes<br>
Train1 Split: 8794 Utterances, 15.1 hours<br>
Train3 Split: 25526 Utterances, 44.7 hours<br>
Train30 Split: 268,392 Utterances, 149.1 hours<br>

sing_300x30x2/dataset/<br>
├── dev/<br>
├───| metadata.csv<br>
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&nbsp;| \<audio files\>.wav<br>
├── dev_spk2gender<br>
├── dev_text<br>
├── test/<br>
├───| metadata.csv<br>
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&nbsp;| \<audio files\>.wav<br>
├── test_spk2gender<br>
├── test_text<br>
├── train1/<br>
├───| metadata.csv<br>
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&nbsp;| \<audio files\>.wav<br>
├── train1_spk2gender<br>
├── train1_text<br>
├── train3/<br>
├───| metadata.csv<br>
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&nbsp;| \<audio files\>.wav<br>
├── train3_spk2gender<br>
└── train3_text<br>
├── train30/<br>
├───| metadata.csv<br>
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&nbsp;| \<audio files\>.wav<br>
├── train30_spk2gender<br>
├── train30_text<br>


# For Colab Training

In [39]:
from google.colab import drive
drive.flush_and_unmount()
#drive.mount('/content/drive')


In [2]:
#
# Curated DAMP 300x30x2 dataset (DSing) unpacked and preprocessed into
# child folder dataset/. DSing is around 1.9GB after being preprocessed.
#

!unzip /content/drive/MyDrive/damp_dataset_20240410.zip -d /

Streaming output truncated to the last 5000 lines.
  inflating: /damp_dataset/train1/M132398937-873755806_1864159-132408394_1590950094-GB-M-035.wav  
  inflating: /damp_dataset/train1/F196863556-586511969_1381576-196863545_1561657911-GB-F-025.wav  
  inflating: /damp_dataset/train1/M754006719-393858919_176664-844902847_1533625120-GB-M-011.wav  
  inflating: /damp_dataset/train1/F822907315-177980987_120356-645804698_1512233575-GB-F-037.wav  
  inflating: /damp_dataset/train1/F443231157-66512068_448309-438276313_1600190617-GB-F-002.wav  
  inflating: /damp_dataset/train1/F715584554-3769646_3769646-1221818620_1633519756-GB-F-010.wav  
  inflating: /damp_dataset/train1/F1377785402-90787007_2368054-40739717_1628488996-GB-F-018.wav  
  inflating: /damp_dataset/train1/M388765895-371824636_190110-388767264_1619183246-GB-M-065.wav  
  inflating: /damp_dataset/train1/M754006719-393858919_176664-844902847_1533625120-GB-M-005.wav  
  inflating: /damp_dataset/train1/F822907315-177980987_120356-6458

In [3]:
!pip install transformers==4.39.1
!pip install accelerate==0.28.0
!pip install evaluate
!pip install torchaudio
!pip install flashlight-text
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 34.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none

# Useful Memory Clearing Tools

In [37]:
del model
del dsing_train
del dsing_val
del data_collator
del asr_pipeline
del training_args
del trainer
del wer_metric

In [38]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

# Basic Setup

In [6]:
import os
import sys
import time
import glob
import re
import json
import random
import tqdm

import torch
import torchaudio
import evaluate

import pandas as pd
import numpy as np
import IPython

from IPython.display import display, HTML, Audio

from datasets import load_dataset, load_metric, ClassLabel, DatasetDict

from torch.utils.data import DataLoader

from torchaudio.models.decoder import ctc_decoder
from torchaudio.models.decoder import download_pretrained_files

from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import pipeline

from transformers import TrainingArguments, Trainer
from transformers.utils import logging




In [33]:
#############################################
# Special Variables...
#############################################
root_folder = "/content/drive/MyDrive"

# Google Colab
#
tokens_file=f"{root_folder}/tokens.txt"
dataset_folder = "/damp_dataset"

# Local
#
# tokens_file="./tokens.txt"
# dataset_folder = "../sing_300x30x2/damp_dataset"
# root_folder = "./"


# read labels for all song utterances for DSING splits (test, dev (aka validation), train1 (aka train)
BATCH_SIZE=4
NUM_EPOCHS=10
MAX_SAMPLES=300 #sample Utterances to use for training.  use -1 to get all the data.
CALC_HOURS=False

In [32]:
#
# Create Pipeline
#

# Simpler, smaller model.  Ou paper, the loss values fine tuned on DSing
# were comparable to the loss values with lv60-self.
#
#model_checkpoint="facebook/wav2vec2-base"
#
# When doing some early testing, I was able to easily overfit to a small
# set of songs which led me to believe that this set of weights likely
# will learn more easily on the sung dataset (DSING).  This specific
# model is used by the author (Ou) and meta fine tuned this model
# on 1000hrs of voice data.
#
model_checkpoint="facebook/wav2vec2-large-960h-lv60-self"

asr_pipeline = pipeline("automatic-speech-recognition", model=model_checkpoint, model_kwargs={"ctc_loss_reduction": 'mean','attention_dropout':0.0,'hidden_dropout':0.0,'feat_proj_dropout':0.0,'mask_time_prob':0.05,'layerdrop':0.0,})
model = asr_pipeline.model

# Used in example here: https://huggingface.co/blog/fine-tune-wav2vec2-english
#model.freeze_feature_encoder()

#
# Gradient Accumulation
# Used in example here: https://huggingface.co/blog/fine-tune-wav2vec2-english
# Per the code: By default, non-reentrant checkpoint stops recomputation as soon as it
#               has computed all needed Tensors.
#
model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})

# Note: I want to use a beam search library that has been pre-trained on librispeech.  the pretrained beam search
#       has been created with all lowercase characters (i.e. phoneme settings, etc.).  I am changing the default
#       model tokenizer vocab settings to accept lower case inputs so i can use the decoder.  However, it will also
#       accept upper case, so this may also impact performance but only for inference.  (i.e. not sure where this
#       is normalized to .
asr_pipeline.tokenizer.do_lower_case = True

target_sampling_rate = asr_pipeline.feature_extractor.sampling_rate

# Count the number of trainable parameters in the model
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Trainable parameters:", trainable_params)


Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.maske

Trainable parameters: 315471520


In [9]:
asr_pipeline.feature_extractor

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

In [10]:
asr_pipeline.tokenizer

Wav2Vec2CTCTokenizer(name_or_path='facebook/wav2vec2-large-960h-lv60-self', vocab_size=32, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	1: AddedToken("<s>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	2: AddedToken("</s>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	3: AddedToken("<unk>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
}

## Decoders

In [11]:
files = download_pretrained_files("librispeech-4-gram")
# Found from Fairseq for Wav2Vec2 -
# https://github.com/facebookresearch/fairseq/blob/main/examples/wav2vec/config/finetuning/vox_960h_2_aws.yaml
# Note: I am not using the same lexicon file though...
# LM_WEIGHT = 2.0
# WORD_SCORE = 0
# SIL_SCORE = -1

LM_WEIGHT = 3.23
WORD_SCORE = -0.26
SIL_SCORE = 0

beam_search_decoder = ctc_decoder(
    lexicon=files.lexicon,
    tokens=tokens_file,
    lm=files.lm,
    nbest=1,
    beam_size=512, #per the paper by Ou
    lm_weight=LM_WEIGHT,
    word_score=WORD_SCORE,
    sil_score=SIL_SCORE,
    blank_token='<pad>',
    unk_word='<unk>'
)

greedy_decoder = ctc_decoder(
    lexicon=files.lexicon,
    tokens=tokens_file,
    lm=files.lm,
    nbest=1,
    beam_size=1,
    lm_weight=LM_WEIGHT,
    word_score=WORD_SCORE,
    sil_score=SIL_SCORE,
    blank_token='<pad>',
    unk_word='<unk>'
)

100%|██████████| 4.97M/4.97M [00:00<00:00, 7.94MB/s]
100%|██████████| 57.0/57.0 [00:00<00:00, 76.4kB/s]
100%|██████████| 2.91G/2.91G [00:07<00:00, 400MB/s]


# Generate Hugginface Dataset Object from DAMP 300x30x2 dataset

Reference: https://huggingface.co/docs/datasets/audio_load

Other pre-processing is heavily inspired from: https://colab.research.google.com/drive/1nCC5Ci-81U5opK_VuXDiZlmcAuATreF2#scrollTo=RBDRAAYxRE6n



# Data Cleaning

In [12]:
def prepare_dataset(batch,tokenizer,feature_extractor):
    """
    Creating a new dataset with the map function to generate the
    keys below.  Padding will occur in the data collator on a per
    batch basis.

    Inputs (i.e. feature extractor):
    input_values   - tensor array for audio samples (shape=(n,) - where n is the number of audio samples)
    attention_mask - used for expressing where there are padded samples

    Outputs (i.e. tokenizer related)
    labels - tensor array for text output tokens (i.e. not transcript).  (shape=(m,) - where m is the number of character tokens)
    """
    chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'
    batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower() + " "

    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct

    # Feature Extractor manipulation
    #
    # this object will return a list of lists because the
    # transcriptions are not padded (i.e. as opposed to a
    # Tensor of tensors when using return_tensors='pt').
    # Padding is done per batch to optimize the size for inference and
    # training.
    #
    # data_collator is responsible for padding the data.
    inputs_values_pt = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"])

    #
    # use this condition because some Wav2Vec2 models require attention_masks
    # for better performance while others do not.
    #
    if "attention_mask" in inputs_values_pt:
        batch["attention_mask"] = inputs_values_pt.attention_mask

    batch["input_values"] = inputs_values_pt.input_values[0]
    batch["input_length"] = len(batch["input_values"])

    # Tokenizer manipulation
    #
    # this object will return a list of lists because the
    # transcriptions are not padded (i.e. as opposed to a
    # Tensor of tensors when using return_tensors='pt').
    # Padding is done per batch to optimize the size for inference and
    # training.
    #
    # data_collator is responsible for padding the data.
    labels_pt = tokenizer(batch["transcription"])
    batch["labels"] = labels_pt['input_ids']

    return batch

In [13]:
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the already tokenized inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).

            Other Options in the pad method that are NOT implemented for this class (i.e. I always want to pad to longest for the
            input and the labels)
            * (not implemented) :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * (not implemented) :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).

    Reference Code here:
    https://huggingface.co/blog/fine-tune-wav2vec2-english


    Note: in the example referenced above, there were parameters for padding max length, etc.  I have created some logic
    in the prepare_dataset to support truncation of data for testing and benchmarking.  I do not think i need max_length
    options for collator at this time.

    """

    tokenizer: Wav2Vec2CTCTokenizer
    feature_extractor: Wav2Vec2FeatureExtractor
    padding: Union[bool, str] = "longest"

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:

        # Features in this case is a list of batch size that contains DataSet objects from the train split
        # (including pretokenized labels). the output batch has been changed from a list back to a dictionary
        # with the respective data objects.
        #
        # Note for future self:
        # pad is being called from PreTrainedTokenizerBase.pad.  From docs:
        #      Pad a single encoded input or a batch of encoded inputs up to predefined length or to the max sequence length
        #      in the batch.
        #
        #    Padding side (left/right) padding token ids are defined at the tokenizer level (with `self.padding_side`,
        #    `self.pad_token_id` and `self.pad_token_type_id`).

        #    Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the
        #    text followed by a call to the `pad` method to get a padded encoding.
        #
        #         <Tip>
        #
        #         If the `encoded_inputs` passed are dictionary of numpy arrays, PyTorch tensors or TensorFlow tensors, the
        #         result will use the same type unless you provide a different tensor type with `return_tensors`. In the case of
        #         PyTorch tensors, you will lose the specific device of your tensors however.
        #
        #         </Tip>

        # Audio Input Data (not tokenized)
        input_features = [{"input_values": feature["input_values"]} for feature in features]

        # batch is a dictionary-like type.
        batch = self.feature_extractor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        # Tokenized Transcript Labels (character level tokens)
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.tokenizer.pad(
            label_features,
            padding=self.padding,
            return_tensors="pt",
        )

        # replace padding with -100 to ignore loss correctly
        batch["labels"] = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        return batch

In [14]:
wer_metric = evaluate.load("wer")

def compute_metrics_dummy(eval_pred):
    return {'wer':1.0}


def compute_metrics(eval_pred,kind='beam',compute=True):
    """
    Calculates WER for a batch of logits and their labels.

    eval_pred - tuple (logit output from the model, token labels from dataset)
    kind - can compare between beam search and greedy search.  both using kenlm

    compute - bool - for training this will compute WER every time its logged.
                     this is nice for understanding if the training is working.
                     for evaluation, this is set to false so compute is run after
                     all batches are processed.

    output is the WER computed from the batch.  if the model is run multiple times, the
    batch WERs are aggregated.

    Note: add_batch and then doing compute will clear the previously cached batch results.
    """
    logits, labels = eval_pred
    #print(f"Logit Type: {type(logits)}")

    # In some scenarios, the input the compute_metrics is a tensor.
    if type(logits) is np.ndarray:
        logits = torch.Tensor(logits)
    else:
        # copy this tensor for computing things...
        logits = logits.clone().detach().requires_grad_(False)
    #print(f"Changing Logit Type to: {type(logits)}")
    #print(f"{logits.shape}")

    if kind=='beam':
        # Creates a list of lists that are of size [batch_size,emissions,vocab_size]
        #
        # Where output[0][0] gives you the CTCHypothesis object.
        #
        # Extract transcript from output[0][0].words (i.e. list of words).
        # May need to join depending on objective.
        #
        predictions = beam_search_decoder(logits)
    elif kind=='greedy':
        # Creates a list of lists that are of size [batch_size,1]
        #
        # Where output[0][0] gives you the CTCHypothesis object.
        #
        # Extract transcript from output[0][0].words (i.e. list of words).
        # May need to join depending on objective.
        #
        predictions = greedy_decoder(logits)
    else:
        print(f"Error passing in decoder kind: {kind}")
        sys.exit()

    ref = asr_pipeline.tokenizer.batch_decode(labels)
    pred = [" ".join(prediction[0].words) for prediction in predictions]

    wer_metric.add_batch(predictions=pred, references=ref)

    if compute:
        return {"wer":wer_metric.compute()}

    return None

# Load Dataset

In [22]:

data_collator = DataCollatorCTCWithPadding(
    tokenizer=asr_pipeline.tokenizer,
    feature_extractor=asr_pipeline.feature_extractor,
)

#
# Only read in MAX_SAMPLES samples that is used to
# get the WER.  This appears to give good performance in the paper (after 10 epochs)
# This has not been shuffled though so it is problematic distribution-wise.
#
dsing_train = load_dataset("audiofolder", data_dir=dataset_folder, split=f'train[:{str(MAX_SAMPLES)}]')
dsing_train = dsing_train.shuffle(seed=44)

if CALC_HOURS:
    arr_lens = [len(d['array']) for d in dsing_train['audio']]
    print(f"Total Hours of Training Data: {np.sum(arr_lens)/ target_sampling_rate / 3600:.2f}")

dsing_train = dsing_train.to_iterable_dataset()
dsing_train = dsing_train.with_format('torch')
# make changes to dataset object to prepare for Wav2Vec2 model
dsing_train = dsing_train.map(
    prepare_dataset,
    remove_columns=["audio","transcription"],
    fn_kwargs={'tokenizer':asr_pipeline.tokenizer, 'feature_extractor':asr_pipeline.feature_extractor}
)

#
# Reading all of validation.  However, this takes about a minute to run,
# so only doing this a few times during each epoch.
#
dsing_val = load_dataset("audiofolder", data_dir=dataset_folder, split='validation')
dsing_val = dsing_val.shuffle(seed=44)
dsing_val = dsing_val.to_iterable_dataset()
dsing_val = dsing_val.with_format('torch')
#
# make changes to dataset object to prepare for Wav2Vec2ForCTC model
#
dsing_val = dsing_val.map(
    prepare_dataset,
    remove_columns=["audio","transcription"],
    fn_kwargs={'tokenizer':asr_pipeline.tokenizer, 'feature_extractor':asr_pipeline.feature_extractor}
)

Resolving data files:   0%|          | 0/8608 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/485 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/483 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/8608 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/485 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/483 [00:00<?, ?it/s]

# Create Training Objects

In [34]:
#logging.set_verbosity_info()
#logger = logging.get_logger("transformers")
#logger.warning("INFO")

training_args = TrainingArguments(
    output_dir=f"/content/drive/MyDrive/test_trainer_300samples",
    overwrite_output_dir = True,
    # Cannot use num_train_epochs because of the lazy loading
    # that IterDatasets require.  They do not actullay know
    # prior to loading how many items are in the dataset
    # so max_steps helps manage that.
    #num_train_epochs=NUM_EPOCHS,
    # typical setup is MAX_SAMPLES / BATCH_SIZE * NUM_EPOCHS
    max_steps=8605 / BATCH_SIZE * NUM_EPOCHS,
    num_train_epochs=NUM_EPOCHS,
    #max_steps=20000,
    # gradient_checkpointing
    # the trade off is  O(sqrt(n)) savings
    # with implemented memory-wise, at the
    # cost of performing one additional
    # forward pass.
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    gradient_accumulation_steps=1,
    #use_cpu=True,
    fp16=True,                      # use when we are doing the GPU based training
    #fp16_full_eval=True,           # use cpu with eval to keep memory footprint low on all GPUs (T4, A100, V100)
    #resume_from_checkpoint=True,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_dir=f"{root_folder}/logs",
    # reduce_lr_on_plateau a proxy for the NewBob technique used by author Ou.
    lr_scheduler_type='reduce_lr_on_plateau',
    # cooldown is a number of steps before attempting to lower the
    # learning rate again.  This is my way of trying to do something like
    lr_scheduler_kwargs={"mode":'min',"factor":0.9,"patience":3, "threshold":0.0025},
    learning_rate=1e-5,              # Based on Ou Paper
    #weight_decay=0.005,
    #warmup_steps=1000,
    save_strategy='steps',
    save_steps=500,
    metric_for_best_model='loss',
    save_total_limit=2,
    report_to='tensorboard',
    logging_strategy='steps',
    logging_steps=100,
    #load_best_model_at_end=True,
    seed=32,  #42 is bad
)

In [35]:
from transformers import TrainerCallback

class MyCallback(TrainerCallback):
    "A callback that prints a message at the beginning of training"

    def on_train_begin(self, args, state, control, **kwargs):
        print("Starting training")

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dsing_train,
    eval_dataset=dsing_val,
    compute_metrics=compute_metrics_dummy,
    tokenizer=asr_pipeline.feature_extractor,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [36]:
start = time.time()
trainer.train()
finish = time.time()
print(f"Finished in {(finish-start)/60} minutes.")

Step,Training Loss,Validation Loss,Wer
500,1.606100,0.724374,1.000000
1000,1.479800,0.689053,1.000000
1500,1.497200,0.679498,1.000000
2000,1.356800,0.635635,1.000000
2500,1.276400,0.618970,1.000000
3000,1.295500,0.663885,1.000000
3500,1.215600,0.610469,1.000000
4000,1.176500,0.609376,1.000000
4500,1.145900,0.643632,1.000000
5000,1.142200,0.591769,1.000000


RuntimeError: [enforce fail at inline_container.cc:595] . unexpected pos 245431616 vs 245431508

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "/content/drive/MyDrive/logs"